In [ ]:
import os
import torch
from huggingface_hub import hf_hub_download
from pipeline_flux import FluxPipeline
from transformer_flux import FluxTransformer2DModel

In [ ]:
bfl_repo="black-forest-labs/FLUX.1-schnell"
device = torch.device('cuda')
dtype = torch.bfloat16
transformer = FluxTransformer2DModel.from_pretrained(bfl_repo, subfolder="transformer", torch_dtype=dtype)
pipe = FluxPipeline.from_pretrained(bfl_repo, transformer=None, torch_dtype=dtype)
pipe.transformer = transformer
pipe.scheduler.config.use_dynamic_shifting = False
pipe.scheduler.config.time_shift = 10
pipe = pipe.to(device)

In [ ]:
if not os.path.exists('ckpt/urae_2k_adapter.safetensors'):
    hf_hub_download(repo_id="Huage001/URAE", filename='urae_2k_adapter.safetensors', local_dir='ckpt', local_dir_use_symlinks=False)
pipe.load_lora_weights("ckpt/urae_2k_adapter.safetensors")

In [ ]:
prompt = "A serene woman in a flowing azure dress, gracefully perched on a sunlit cliff overlooking a tranquil sea, her hair gently tousled by the breeze. The scene is infused with a sense of peace, evoking a dreamlike atmosphere, reminiscent of Impressionist paintings."
height = 2048
width = 2048
image = pipe(
    prompt,
    height=height,
    width=width,
    guidance_scale=0,
    num_inference_steps=4,
    max_sequence_length=256,
    generator=torch.manual_seed(8888),
    ntk_factor=10,
    proportional_attention=True
).images[0]
image